# PyTorch

Here's your best friend when working with PyTorch: https://pytorch.org/docs/stable/index.html.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D, art3d
from matplotlib.patches import Circle, Ellipse
import seaborn as sns
import matplotlib.ticker as ticker
from IPython.display import Markdown, display, HTML

import torch
import torch.nn as nn
import torch.optim as optim

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

## PyTorch basic operations tasks

**Task 1.** Calculate the sigmoid (logistic) function on every element of the following array [0.3, 1.2, -1.4, 0.2, -0.1, 0.1, 0.8, -0.25] and print the last 5 elements. Use only tensor operations.

In [2]:
x = torch.tensor( [0.3, 1.2, -1.4, 0.2, -0.1, 0.1, 0.8, -0.25] )
x = torch.sigmoid(x)
print(x[:5])


tensor([0.5744, 0.7685, 0.1978, 0.5498, 0.4750])


**Task 2.** Calculate the dot product of the following two vectors:<br/>
$x = [3, 1, 4, 2, 6, 1, 4, 8]$<br/>
$y = [5, 2, 3, 12, 2, 4, 17, 9]$<br/>
a) by using element-wise mutliplication and torch.sum,<br/>
b) by using torch.dot,<br/>
b) by using torch.matmul and transposition (x.T).

In [9]:
x = torch.tensor([3,1,4,2,6,1,4,8])
y = torch.tensor([5,2,3,12,2,4,17,9])

a = torch.sum(x * y)
b = torch.dot(x,y)
y_t = torch.t(y)
c = torch.matmul(x,y_t)

print(a)
print(b)
print(c)

tensor(209)
tensor(209)
tensor(209)


**Task 3.** Calculate the following expression<br/>
$$\frac{1}{1 + e^{-x_0 \theta_0 - \ldots - x_9 \theta_9 - \theta_{10}}}$$
for<br/>
$x = [1.2, 2.3, 3.4, -0.7, 4.2, 2.7, -0.5, 1.4, -3.3, 0.2]$<br/>
$\theta = [1.7, 0.33, -2.12, -1.73, 2.9, -5.8, -0.9, 12.11, 3.43, -0.5, -1.65]$<br/>
and print the result. Use only tensor operations.

In [22]:
x = torch.tensor([1.2,2.3,3.4,-0.7,4.2,2.7,-0.5,1.4,-3.3,0.2])
theta = torch.tensor([1.7,0.33,-2.12,-1.73,2.9,-5.8,-0.9,12.11,3.43,-0.5,-1.65])
y = 1/(1 + torch.exp(torch.dot(-x,theta[:-1])-theta[-1:]))
print(y.item())


0.0876237079501152


# Tensor gradients

**Task 4.** Calculate the derivative $f'(w)$ using PyTorch and backward propagation (the backward method of the Tensor class) for the following functions and points:
  - $f(w) = w^3 + w^2$ and $w = 2.0$,
  - $f(w) = \text{sin}(w)$ and $w = \pi$,
  - $f(w) = \ln(w * e^{3w})$ and $w = 1.0$.
  
Print the values of those derivatives.

In [29]:
x = torch.tensor(2.0, requires_grad = True)
y = x ** 3 + x ** 2
y.backward()
print(float(x.grad.data))

torch.pi = torch.acos(torch.zeros(1)).item() * 2
x = torch.tensor(torch.pi, requires_grad = True)
y = torch.sin(x)
y.backward()
print(float(x.grad.data))

x = torch.tensor(1.0, requires_grad = True)
y = torch.log(x * torch.exp(3*x))
y.backward()
print(float(x.grad.data))

16.0
-1.0
4.0


**Task 5.** Calculate the derivative $\frac{\partial f}{\partial w_1}(w_1, w_2, w_3)$ using PyTorch and backward propagation (the backward method of the Tensor class) for the following functions and points:
  - $f(w_1, w_2) = w_1^3 + w_1^2 + w_2$ and $(w_1, w_2) = (2.0, 3.0)$,
  - $f(w_1, w_2, w_3) = \text{sin}(w_1) * w_2 + w_1^2 * w_3$ and $(w_1, w_2) = (\pi, 2.0, 4.0)$,
  - $f(w_1, w_2, w_3) = e^{w_1^2 + w_2^2 + w_3^2} + w_1^2 + w_2^2 + w_3^2$ and $(w_1, w_2, w_3) = (0.5, 0.67, 0.55)$.
  
Print the values of those derivatives.

In [36]:
w1 = torch.tensor(2.0, requires_grad = True)
w2 = torch.tensor(3.0, requires_grad = True)
y = w1 ** 3 + w1 ** 2 + w2
y.backward()
print(float(w1.grad.data))

torch.pi = torch.acos(torch.zeros(1)).item() * 2
w1 = torch.tensor(torch.pi, requires_grad = True)
w2 = torch.tensor(2.0, requires_grad = True)
w3 = torch.tensor(4.0, requires_grad = True)
y = torch.sin(w1)*w2 + w1 ** 2 * w3
y.backward()
print(float(w1.grad.data))

w1 = torch.tensor(0.5, requires_grad = True)
w2 = torch.tensor(0.67, requires_grad = True)
w3 = torch.tensor(0.55, requires_grad = True)
y = torch.exp(w1**2+w2**2+w3**2)+w1**2+w2**2+w3**2
y.backward()
print(float(w1.grad.data))

16.0
23.132741928100586
3.722090005874634


# Backpropagation

**Task 6.** Write the calculate_se_array(base_w, x, y_target, weight_idx, weight_start, weight_end) which generates a numpy array of squared errors for the following sigmoid function $\text{sigmoid}(w * x)$ and target value $y_target$, where $x$ is a vector of dimension 3 and $w$ is a vector of weights of the same dimension. The function should generate a set of weight vectors w which have the same value as base_w on all coordinates other than weight_idx, and should have 100 values on the weight_idx coordinate ranging from weight_start to weight_end. Example for weight_idx =1: 

w = [[base_w[0], weight_start, base_w[2]], [base_w[0], weight_start + step, base_w[2]], ..., [base_w[0], weight_end - step, base_w[2]], [base_w[0], weight_end, base_w[2]]]

Then the function should calculate and return an array of squared errors between the value of the sigmoid on x with respect to all those weight vectors and the target value y_target. This will allow you to see how the error of a single neuron depends on every weight separately.

The function should work even if base_w and x are tensors (a safe way to do is tensor.cpu().detach().numpy()).

You're encouraged to experiment with different neural nets to see how the error depends on all its parameters.

In [ ]:
def calculate_se_array(base_w, x, y_target, weight_idx, weight_start, weight_end):
    ########################
    # Write your code here #
    ########################


In the next cell you can test the method and in the cell after that you can see the error plots for the training you saw in the lecture. The red dot indicates the current weight value and the error it gives. The red dot should arrive at the lowest point at the end of training.

In [ ]:
x = torch.tensor([2., -1., 3.], requires_grad=False)
w = torch.tensor([-0.65, -0.4, -1.], requires_grad=True)
y_target = 0.65

base_w = w
weight_idx = 0 
weight_start = -5
weight_end = 5

print(calculate_se_array(base_w, x, y_target, weight_idx, weight_start, weight_end))

In [ ]:
x = torch.tensor([2., -1., 3.], requires_grad=False)
w = torch.tensor([-0.65, -0.4, -1.], requires_grad=True)
y_target = 0.65

optimizer = optim.SGD([w], lr=0.1)

losses = []
n_epochs = 100
for epoch in range(n_epochs):

    optimizer.zero_grad()
    y = torch.sigmoid(torch.sum(x * w))
    loss = torch.pow(y - y_target, 2)
    loss.backward()
    losses.append(loss.item())
    optimizer.step()

    if epoch > 0 and epoch % 10 == 0:
        print("Epoch: {}".format(epoch))
        print(w.cpu().detach().numpy())
        print(loss.item())
        for i in range(3):    
            w_range = np.linspace(-5, 5, 100)
            errors = calculate_se_array(w, x, y_target, i, -5, 5)
            sns.lineplot(x=w_range, y=errors).set_title('Training loss')
            plt.scatter([w[i].item()], [loss.item()], color='red')
            plt.xlabel("weight {}".format(i))
            plt.ylabel("squared loss")
            plt.show()

**Task 7*.** Train a neural network with:
  - two input neurons, 
  - four hidden neurons with no bias and sigmoid activation in the first hidden layer,
  - four hidden neurons with no bias and with sigmoid activation in the second hidden layer,
  - one output neuron with no bias and without sigmoid activation 
  
to get a good approximation of $f(x) = x_1 * x_2 + 1$ on the following dataset $D = \{(1.0, 1.0), (0.0, 0.0), (2.0, -1.0), (-1.0, 0.5), (-0.5, -2.0)\}$, i.e. the network should satisfy:
  - $\text{net}(1.0, 1.0) \sim 2.0$,
  - $\text{net}(0.0, 0.0) \sim 1.0$,
  - $\text{net}(2.0, -1.0) \sim -1.0$,
  - $\text{net}(-1.0, 0.5) \sim 0.5$,
  - $\text{net}(-0.5, -2.0) \sim 2.0$.
  
Use seed=6789 to initialize the network.

After training print all weights and separately print $w_{1, 2}^{(1)}$ (the weight from the second input to the first hidden neuron in the first hidden layer) and $w_{1, 3}^{(3)}$ (the weight from the third hidden neuron in the second hidden layer to the output unit).

Print the values of the network on the training points and verify that these values are closer to the real values of the $f$ function than $\epsilon = 0.1$, i.e. $|\text{net}(x) - f(x)| < \epsilon$ for $x \in D$.

Because this network is only tested on the training set, it will certainly overfit if trained long enough. Train for 1000 epochs and then calculate
  - $\text{net}(2.0, 2.0)$,
  - $\text{net}(-1.0, -1.0)$,
  - $\text{net}(3.0, -3.0)$.
  
How far are these values from real values of the function $f$?

In [ ]:
class TwoLayerModel(nn.Module):
    def __init__(self, seed):
        super().__init__()

        self.seed = torch.manual_seed(seed)

        ########################
        # Write your code here #
        ########################


    def forward(self, x):
        ########################
        # Write your code here #
        ########################


        return x

In [ ]:
# Initialize the data

########################
# Write your code here #
########################


# Initialize the neural network and optimizer

########################
# Write your code here #
########################


# Perform the training for 1000 epochs

########################
# Write your code here #
########################


print("All weights - layer by layer starting from left to right")

########################
# Write your code here #
########################

    
print()
print("w_{{1, 2}}^{{(1)}} (rounded to 4 decimal points)")

########################
# Write your code here #
########################


print()
print("w_{{1, 3}}^{{(2)}} (rounded to 4 decimal points)")

########################
# Write your code here #
########################


print()
print("Values on the training data (verify the difference is below epsilon)")

########################
# Write your code here #
########################


print()
print("NN value on x=[2.0, 2.0] vs real value")

########################
# Write your code here #
########################


print()
print("NN value on x=[-1.0, -1.0] vs real value")

########################
# Write your code here #
########################

print()
print("NN value on x=[3.0, -3.0] vs real value")

########################
# Write your code here #
########################
